<center>
<table>
  <tr>
    <td><img src="http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://github.com/astg606/py_materials/blob/master/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h1><font color= "blue" size="+3">ASTG Python Courses</font></h1>
</center>

---

<center><h1> <font color="red">Reading MODIS hdf Files using pyhdf</font></h1></center>

This Jupyter notebook shows an example of how to use the **pyhdf**, **Numpy**, **Matplotlib**, and **Cartopy** Python packages to work with MODIS files in HDF4 format.  

The main workflow steps are:
- Open a MODIS HDF4 file
- Read the global file metadata
    - Recognize the file attribute
    - Find names of variables and their attributes
- Read dataset from file
- Visualize satellite data on a map

Write an application a read a collection of MODIS data files and plot a specific field within a specified region.

## <font color="red">Primary References/Resources</font>

- [Moderate Resolution Imaging Spectrometer (MODIS)](https://modis.gsfc.nasa.gov/data/)
- [HDF-EOS Comprehensive Examples page](http://hdfeos.org/zoo/)
- [How to read a MODIS HDF4 file using python and pyhdf ?](https://moonbooks.org/Articles/How-to-read-a-MODIS-HDF-file-using-python-/)
- [SD (scientific dataset) API (pyhdf.SD)](http://fhs.github.io/pyhdf/modules/SD.html) 

## <font color="red">Import the Python Packages</font>

Four Python packages (libraries)  used in this Notebook:
- **pyhdf**: Read HDF4 files
- **NumPy**: Perform array operations
- **Matplotlib**: Make static plots (mainly two-dimensional)
- **Cartopy**: Create maps

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import pprint
import glob

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.xarray
import cartopy
from cartopy import crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
from pyhdf.SD import SD
from pyhdf.SD import SDS
from pyhdf.SD import SDC
from pyhdf.SD import SDim
from pyhdf.SD import SDAttr

In [ ]:
print(f"Version of Numpy:   {np.__version__}")
print(f"Version of Cartopy: {cartopy.__version__}")
print(f"Version of Xarray:  {xr.__version__}")

## <font color="red">[MODIS](https://modis.gsfc.nasa.gov/)</font>

#### MODIS Intrument
MODIS (or Moderate Resolution Imaging Spectroradiometer) is a key instrument aboard the Terra and Aqua satellites. Terra's orbit around the Earth is timed so that it passes from north to south across the equator in the morning, while Aqua passes south to north over the equator in the afternoon. Terra MODIS and Aqua MODIS are viewing the entire Earth's surface every 1 to 2 days, acquiring data in 36 spectral bands, or groups of wavelengths. These data will improve our understanding of global dynamics and processes occurring on the land, in the oceans, and in the lower atmosphere. MODIS is playing a vital role in the development of validated, global, interactive Earth system models able to predict global change accurately enough to assist policy makers in making sound decisions concerning the protection of our environment.

#### MODIS Data

The MODIS instrument has a viewing swath width of 2,330 km and views the entire surface of the Earth every one to two days. Its detectors measure 36 spectral bands between 0.405 and 14.385 µm, and it acquires data at three spatial resolutions -- 250m, 500m, and 1,000m.

The many data products derived from MODIS observations describe features of the land, oceans and the atmosphere that can be used for studies of processes and trends on local to global scales. MODIS products are available from several sources.  

- [MODIS Level 1](http://ladsweb.nascom.nasa.gov/) and atmosphere products are available through the LAADS web.  
- [Land Products](https://lpdaac.usgs.gov/) are available through the Land Processes DAAC at the U. S. Geological Survey EROS Data Center (EDC).  
- [Cryosphere data products](http://nsidc.org/daac/modis/index.html) (snow and sea ice cover) are available from the National Snow and Ice Data Center (NSIDC) in Boulder, Colorado.  
- [Ocean color products and sea surface temperature products](http://oceancolor.gsfc.nasa.gov/) along with information about these products are obtainable at the OCDPS at GSFC.  

Users with an appropriate x-band receiving system may capture regional data directly from the spacecraft using the MODIS Direct Broadcast signal. 

#### MODIS File Naming Conventions
- [MODIS Naming Conventions](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/modis-overview/#:~:text=MODIS%20Filenames,A2019159.)
- [MODIS Level-2 Hierarchical Data Format (HDF)](https://modis-images.gsfc.nasa.gov/MOD07_L2/filename.html)
- [MODIS/VIIRS Land Product Subsets](https://modis.ornl.gov/documentation.html)


MODIS filenames follow a naming convention which gives useful information regarding the specific product. The filename `MOD09A1.A2006001.h08v05.005.2006012234657.hdf` indicates:

- `MOD09A1`: Product Short Name
- `A2006001`: Julian Date of Acquisition (A-YYYYDDD)
- `h08v05`: Tile Identifier (horizontalXXverticalYY)
- `005`: Collection Version
- `2006012234567`: Julian Date of Production (YYYYDDDHHMMSS)
- `hdf`: Data Format (HDF-EOS)

## <font color="red"> Accessing a Sample HDF4 Data Files</font>

### <font color="blue"> Step 1: Identify the Location of the File</font>

Directory where the MODIS files are located:

In [ ]:
data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/MODIS_Data/"
#data_dir = "/tljh-data/sat_data/MODIS_Data"

Full path to the file names:

In [ ]:
file_name = os.path.join(data_dir, "MOD021KM.A2015013.1240.006.2015014140954.hdf")
geo_file_name = os.path.join(data_dir, "MOD03.A2015013.1240.006.2015013194359.hdf")

Name of the field of interest:

In [ ]:
field_name = "EV_Band26"

### <font color="blue"> Step 2: Open the File</font>

Opening files for reading:

In [ ]:
fid = SD(file_name, SDC.READ)

### <font color="blue"> Step 3: Obtain the File Attributes</font>

Basic information on the files:

- The first number indicates the number of datasets in the file (not to be confused w/ xarray datasets)
- The second number indicates the number of attributes attached to the global file.

In [ ]:
fid.info()

#### File Attributes

- We can access the file attributes which hold important global metadata.

In [ ]:
file_attrs = fid.attributes()
pprint.pprint(file_attrs)

In [ ]:
for idx, name in enumerate(file_attrs.keys(), start=1):
    print(f"{idx:>3} --> {name}: \n\t {file_attrs[name]}")

We can also access the datasets' names and basic info such as shape and dimension labels

In [ ]:
file_dts = fid.datasets()
for index, name in enumerate(file_dts.keys(), start=1):
    print(f"{index:>3}- {name}")

In [ ]:
file_dts[field_name]

### <font color="blue">Step 4: Extract the Dataset</font>

Let's assume that we want to extract data from the field `EV_Band26`.

The `select()` method from the `SD` class allows us to extract a dataset (object) given it's name or index number.

In [ ]:
sample_ds = fid.select(field_name)

Get basic information from the dataset:

- The `info()` function in the `SDS` class allows us to get the dataset name, rank, dimension lengths, data type, and number of attributes.

In [ ]:
sample_ds.info()

List the dataset attributes:

In [ ]:
attrs = sample_ds.attributes()
attrs

#### Extract the data

- We can retrieve and store the data itself as a NumPy array using the `get()` function.

In [ ]:
sample_data = sample_ds.get()

Confirms that the data has been stored as a NumPy array.

In [ ]:
print("Dataset Class Type: ", type(sample_data))

Just like any NumPy array, we can get the shape and dtype.

In [ ]:
sample_data.shape

In [ ]:
sample_data.dtype

We need to change the type from integer to float:

In [ ]:
sample_data = sample_data.astype(np.double)

### <font color="blue">Step 6: Restore the Data</font>

### Get the dataset attributes

In [ ]:
attrs = sample_ds.attributes(full=1)

long_name = attrs["long_name"][0]
add_offset = attrs["radiance_offsets"][0]
_FillValue = attrs["_FillValue"][0]
scale_factor = attrs["radiance_scales"][0]       
valid_min = attrs["valid_range"][0][0]        
valid_max = attrs["valid_range"][0][1]        
units = attrs["radiance_units"][0]

### Use the attributes to restore the data

In [ ]:
def restore_data(data, scale_factor, add_offset,  _FillValue, 
                 valid_min, valid_max):
    """
       Use the attributes to:
        1- Select the values within the valid range
        2- Mask the filled values
        3- To apply the offset and scale to the data
    """
    invalid = np.logical_or(data > valid_max, data < valid_min)
    invalid = np.logical_or(invalid, data == _FillValue)
    data[invalid] = np.nan
    data -= add_offset
    data *= scale_factor
    data = np.ma.masked_array(data, np.isnan(data))
    return data

In [ ]:
data = restore_data(sample_data, 
                    scale_factor, add_offset,  
                    _FillValue, 
                    valid_min, valid_max)

In [ ]:
print(data.min(), data.max())

#### View the data as an image

In [ ]:
fig, ax = plt.subplots (figsize = (8,8))
im = ax.imshow(data)
plt.colorbar(im)

### <font color="blue">Step 7: Read Geolocation Dataset from MOD03 Product</font>

In [ ]:
geo_fid = SD(geo_file_name, SDC.READ)

In [ ]:
geo_fid.info()

In [ ]:
geo_file_attrs = geo_fid.attributes()
for index, name in enumerate(geo_file_attrs.keys(), start=1):
    print(index, name)

In [ ]:
geo_file_dts = geo_fid.datasets()
for index, name in enumerate(geo_file_dts.keys(), start=1):
    print(index, name)

In [ ]:
lats = geo_fid.select('Latitude').get()
lats.shape

In [ ]:
lons = geo_fid.select('Longitude').get()
lons.shape

Find middle location:

In [ ]:
lat_m = np.nanmean(lats)
lon_m = np.nanmean(lons) 

### <font color="blue">Step 8: Use Cartopy to Plot the Data</font>

The reference plot can be seen at:

[https://hdfeos.org/zoo/LAADS/MOD021KM.A2015013.1240.006.2015014140954.hdf.py.png](https://hdfeos.org/zoo/LAADS/MOD021KM.A2015013.1240.006.2015014140954.hdf.py.png)

In [ ]:
#map_projection = ccrs.PlateCarree()
map_projection = ccrs.LambertAzimuthalEqualArea(
                      central_longitude=lon_m, 
                      central_latitude=lat_m
                )
data_transform = ccrs.PlateCarree()

In [ ]:
#Create the figure object with the dimansion of the figure
subplot_kw = dict(projection=map_projection)
fig, ax = plt.subplots(1, 1,
                       figsize=(12, 9),
                       subplot_kw=subplot_kw)

im = ax.pcolormesh(lons, lats, data, transform=data_transform)

# Map features
map_res = '110m'
ax.coastlines(resolution=map_res, linewidth=1.0)
ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1.0)

# Colorbar
cbar = fig.colorbar(im, ax=ax,  orientation="vertical", shrink=0.95)
cbar.ax.tick_params(labelsize=15)
cbar.set_label(units, labelpad=+1)

# ---> Ticks and labels
gl = ax.gridlines(
    draw_labels=True, 
    linewidth=2, color='gray', 
    alpha=0.5, linestyle='--'
)
gl.xlabels_top = False
gl.ylabels_right = False

## <font color="red">Application</font>

- Takes a collection of MODIS data files
- Select a field (contained in the files)
- Loop over the files:
     - Select only the files which horizontal coverage falls within a prescribed latitude range
     - Perform a global plot on the selected field

#### Simple Test to Create a Global Map

In [ ]:
map_projection = ccrs.PlateCarree()
data_transform = ccrs.PlateCarree()

In [ ]:
def map_globe(map_res, map_projection):
    """
    Map the entire globe, draw continents/countries, 
    longitude/latitude lines.
    Paramters
    ---------
    map_res : str
        Resolution ('110m', '50m', '10m') of the Cartopy features.
    Returns
    -------
    fig, ax : Matplotlib figure and axes
    """
    subplot_kw = dict(projection=map_projection)
    fig, ax = plt.subplots(1, 1,
                       figsize=(15, 9),
                       subplot_kw=subplot_kw)

    # Consider the entire globe
    ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())

    # Map features
    #map_res = '110m'
    ax.coastlines(resolution=map_res, linewidth=1.0)
    ax.add_feature(cfeature.LAND, edgecolor='black', linewidth=1.0)
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1.0)

    # ---> Ticks and labels
    gl = ax.gridlines(crs=map_projection, draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False

    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    return fig, ax

In [ ]:
map_res = '110m'
fig, ax = map_globe(map_res, map_projection)

#### Field of interest:

In [ ]:
DATAFIELD_NAME = 'LST'

#### Latidude range of the area of interest:

In [ ]:
min_lat = -55.0
max_lat =  60.0

#### Get the list of possible files to process

In [ ]:
list_files = glob.glob(data_dir+'MOD11_*.hdf')

In [ ]:
len(list_files)

In [ ]:
list_files

Each of the data file (starting with `MOD11`) comes if its corresponding geolocation file (starting with  `MOD03`).

In [ ]:
list_geo_files = list()
for fname in list_files:
    basename = os.path.basename(fname)
    geo_fname = glob.glob(data_dir+'MOD03.'+basename[9:26]+'*hdf')[0]
    list_geo_files.append(geo_fname)    

In [ ]:
len(list_geo_files)

In [ ]:
list_geo_files

In [ ]:
for fname, geo_fname in zip(list_files, list_geo_files):
    print(f"{os.path.basename(fname)} -> {os.path.basename(geo_fname)}")

#### Plot the data

In [ ]:
map_res = '110m'
fig, ax = map_globe(map_res, map_projection)

# Loop over the files and do the plot:
for fname, geo_fname in zip(list_files, list_geo_files):
    hdf_geo = SD(geo_fname, SDC.READ)
    
    # Extract the latitude data
    latitude = hdf_geo.select('Latitude')
    lat = latitude[:,:]

    # Only consider the the data file which horizontal coverage 
    # overlaps with the prescribed latitude rangnge
    if np.min(lat)<min_lat or np.max(lat)>max_lat:
        print(f"Rejecting the files: \n\t {os.path.basename(fname)} \n\t {os.path.basename(geo_fname)}")
        continue  # Move to the next file

    longitude = hdf_geo.select('Longitude')
    lon = longitude[:,:]

    print(f"Processing the files: \n\t {os.path.basename(fname)} \n\t {os.path.basename(geo_fname)}")

    # Read dataset.
    hdf = SD(fname, SDC.READ)

    data2D = hdf.select(DATAFIELD_NAME)
    data = data2D[:,:].astype(np.double)

    # Get the dataset attributes
    attrs = data2D.attributes(full=1)
    long_name = attrs["long_name"][0]
    add_offset = attrs["add_offset"][0]
    _FillValue = attrs["_FillValue"][0]
    scale_factor = attrs["scale_factor"][0]
    valid_min = attrs["valid_range"][0][0]
    valid_max = attrs["valid_range"][0][1]
    units = attrs["units"][0]

    data = restore_data(data, scale_factor, add_offset,  _FillValue,
                        valid_min, valid_max)

    # Plot the data
    im = ax.pcolormesh(lon, lat, data, transform=map_projection)
    
# Add a colorbar
cbar = fig.colorbar(im, ax=ax,  orientation="horizontal", shrink=0.75)
cbar.ax.tick_params(labelsize=15)
cbar.set_label(units, labelpad=+1)

## <font color="red">L4 Level File</font>


In [ ]:
file_name_L3 = os.path.join(data_dir, 
                        "MOD08_D3.A2020126.061.2020127085602.hdf")

In [ ]:
fid = SD(file_name_L3, SDC.READ)

In [ ]:
fid.info()

In [ ]:
for idx, name in enumerate(file_attrs.keys(), start=1):
    print(f"{idx:>3} --> {name}: \n\t {file_attrs[name]}")

In [ ]:
file_dts = fid.datasets()
for index, name in enumerate(file_dts.keys(), start=1):
    print(f"{index:>3}- {name}")

#### Latitude and Longitude Grid Points

In [ ]:
lons = fid.select("XDim").get()
lats = fid.select("YDim").get()

#### Extract `Cirrus_Fraction_Infrared`

In [ ]:
field_name = "Cirrus_Fraction_Infrared"

In [ ]:
cfi_ds = fid.select(field_name)

In [ ]:
def get_ds_attribute_value(sample_ds, attr_name):
    '''
    Obtain the value of a specified attribute in a dataset.
    
    Parameter
    ---------
    sample_ds : SDS object
    attr_name : str
         Attribute name    
    
    Returns
    --------
    value: float, int, str, list
         Value of the attribute. If attribute not available, None.
    '''
    for key, value in sample_ds.attributes().items():
        if key == attr_name:
            return value 
    return None

In [ ]:
def restore_data(sample_ds):
    '''
    Restore the data given the dataset attribute
    Parameter(s): SDS object
    Return Type(s): NumPy array
    Function: restores data of a given dataset object
    '''
    _FillValue = get_ds_attribute_value(sample_ds, '_FillValue')
    scale_factor = get_ds_attribute_value(sample_ds, 'scale_factor')
    add_offset = get_ds_attribute_value(sample_ds, 'add_offset')
    valid_range = get_ds_attribute_value(sample_ds, 'valid_range')
    
    data = sample_ds.get().astype('float')
    
    if valid_range:
        valid_min, valid_max = valid_range[0], valid_range[1]
        invalid = np.logical_or(data > valid_max, data < valid_min)
    
    #data[invalid] = np.nan
    #data = np.where(data != _FillValue, data, np.nan)
    
    if _FillValue:
        if valid_range:
            invalid = np.logical_or(invalid, data == _FillValue)
            data[invalid] = np.nan
        else:
            data = np.where(data != _FillValue, data, np.nan)
    
    if add_offset:
        data -= add_offset
    if scale_factor:
        data *= scale_factor
    data = np.ma.masked_array(data, np.isnan(data))
    return data

In [ ]:
data = restore_data(cfi_ds)

#### Map `Cirrus_Fraction_Infrared`

In [ ]:
units = get_ds_attribute_value(cfi_ds, 'units')
long_name = get_ds_attribute_value(cfi_ds, 'long_name')

In [ ]:
map_projection = ccrs.PlateCarree()
data_transform = ccrs.PlateCarree()

In [ ]:
subplot_kw = dict(projection=map_projection)
fig, ax = plt.subplots(1, 1,
                       figsize=(12, 9),
                       subplot_kw=subplot_kw)

im = ax.pcolormesh(lons, lats, data, transform=data_transform, cmap="hsv")

# Map features
map_res = '110m'
ax.coastlines(resolution=map_res, linewidth=1.0)
#ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1.0)

# Colorbar
cbar = fig.colorbar(im, ax=ax,  orientation="vertical", shrink=0.55)
cbar.ax.tick_params(labelsize=15)
cbar.set_label(units, labelpad=+1)

# ---> Ticks and labels
gl = ax.gridlines(
    draw_labels=True, 
    linewidth=2, color='gray', 
    alpha=0.5, linestyle='--'
)
gl.xlabels_top = False
gl.ylabels_right = False
ax.set_title(long_name, fontsize=10);